In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_06 import *

## ConvNet

Let's get the data and training interface from where we left in the last notebook.

In [3]:
x_train, y_train, x_valid, y_valid = get_data()
x_train, x_valid = normalize_to(x_train, x_valid)

train_ds = Dataset(x_train, y_train)
valid_ds = Dataset(x_valid, y_valid)

nh = 50
bs = 512
c = y_train.max().item() + 1

loss_func = F.cross_entropy
data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [4]:
mnist_view = view_tfm(1, 28, 28)
cbfs = [Recorder,
       partial(AvgStatsCallback, accuracy),
       CudaCallback,
       partial(BatchTransformXCallback, mnist_view)]

In [5]:
nfs = [8, 16, 32, 64, 64]

In [6]:
learn, run = get_learn_run(nfs, data, 0.4, conv_layer, cbs=cbfs)

In [7]:
%%time
run.fit(2, learn)

train: [1.50970578125, tensor(0.4919, device='cuda:0')]
valid: [0.42316279296875, tensor(0.8603, device='cuda:0')]
train: [0.26411359375, tensor(0.9170, device='cuda:0')]
valid: [0.19294384765625, tensor(0.9392, device='cuda:0')]
CPU times: user 3.09 s, sys: 428 ms, total: 3.51 s
Wall time: 3.12 s


## Batchnorm

Let's start by building our own `BatchNorm` layer from scratch.

In [8]:
class BatchNorm(nn.Module):
    
    def __init__(self, nf, mom=0.1, eps=1e-5):
        super.__init__()
        self.mom = mom
        self.eps = eps
        self.mults = nn.Parameter(torch.ones(nf, 1, 1))
        self.adds = nn.Parameter(torch.zeros(nf, 1, 1))
        self.register_buffer('vars', torch.ones(1, nf, 1, 1))
        self.register_buffer('means', torch.ones(1, nf, 1, 1))
        
    def update_stats(self, x):
        m = x.mean((0,2,3), keepdim=True)
        v = x.var((0,2,3), keepdim=True)
        self.means.lerp_(m, self.mom)
        self.vars.lerp_(v, self.mom)
        return m, v
    
    def forward(self, x):
        

In [17]:
a = torch.arange(10).float()
a

tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [29]:
b = torch.empty([10]).fill_(1)
a_lin_interpolation = a.lerp(b, 0.9)
a_lin_interpolation

tensor([0.9000, 1.0000, 1.1000, 1.2000, 1.3000, 1.4000, 1.5000, 1.6000, 1.7000,
        1.8000])

In [30]:
def lin_interp(x, y, mom):
    for s, e in zip(x, y):
        m1 = s + mom * (e - s)
        print(m1)
    return m1
lin_interp(a, b, 0.9)

tensor(0.9000)
tensor(1.)
tensor(1.1000)
tensor(1.2000)
tensor(1.3000)
tensor(1.4000)
tensor(1.5000)
tensor(1.6000)
tensor(1.7000)
tensor(1.8000)


tensor(1.8000)